In [1]:
!pip -q install transformers==4.17
!pip -q install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 26.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00


In [2]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
# class OneHotTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         outputs = model(**inputs)
#         logits = outputs.get("logits")

#         # Use BCEWithLogitsLoss for multi-label classification
#         loss_fct = nn.BCEWithLogitsLoss()
#         loss = loss_fct(logits, labels.float())

#         return (loss, outputs) if return_outputs else loss

class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            # Handle the case where logits are missing, e.g., raise an exception or return a default loss
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Optional compute_metrics function to get the accuracy, precision, recall, f1 for each of the individual labels
# without averaging them

# Function for individual metrics of each category
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(pred):
#     # Get predictions and apply sigmoid to get probabilities
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)  # Threshold at 0 to get binary predictions

#     # Get true labels
#     labels = pred.label_ids

#     # Calculate precision, recall, and f1-score for each label without averaging
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
#     # Calculate accuracy per label (how many times each label is correctly predicted)
#     accuracy_per_label = (preds == labels).mean(axis=0)

#     # Prepare a dictionary to store metrics per category
#     metrics = {
#         'accuracy_per_label': accuracy_per_label,
#         'precision_per_label': precision,
#         'recall_per_label': recall,
#         'f1_per_label': f1
#     }

#     # Optionally, if you want to calculate an overall accuracy:
#     overall_accuracy = (preds == labels).all(axis=1).mean()  # This gives the fraction of samples where all labels are correctly predicted
#     metrics['overall_accuracy'] = overall_accuracy

#     return metrics

In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-10 05:12:17,501] A new study created in memory with name: no-name-14b46f77-2337-4ff1-a5ca-171de121352e
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.345750,0.872527,0.297714,0.269360,0.282828
2,No log,0.292937,0.890110,0.508001,0.404040,0.448174
3,No log,0.275737,0.889011,0.711514,0.498316,0.525632
4,0.307100,0.258353,0.909341,0.729670,0.599327,0.650666
5,0.307100,0.235677,0.918681,0.755916,0.632997,0.680379
6,0.307100,0.230435,0.919231,0.736315,0.656566,0.692362


***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in label

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-10 05:17:46,255] Trial 0 finished with value: 0.6923624070287797 and parameters: {'learning_rate': 1.4483294901093566e-05, 'batch_size': 4, 'num_train_epochs': 6, 'weight_decay': 0.04447297105876698}. Best is trial 0 with value: 0.6923624070287797.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featur

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.228701,0.917582,0.722801,0.656566,0.686918
2,No log,0.227937,0.919231,0.725958,0.666667,0.693697
3,No log,0.227796,0.918681,0.727512,0.666667,0.692456
4,No log,0.222773,0.921429,0.739366,0.666667,0.699028
5,No log,0.222029,0.920330,0.735411,0.666667,0.696927
6,No log,0.221035,0.921429,0.739373,0.673401,0.701496
7,No log,0.219735,0.923077,0.738901,0.676768,0.704803
8,0.164800,0.219601,0.921978,0.733146,0.676768,0.702068
9,0.164800,0.219004,0.923077,0.738901,0.676768,0.704803


...

[I 2024-12-10 06:09:00,315] Trial 8 finished with value: 0.7859693508182166 and parameters: {'learning_rate': 1.7496859314585534e-06, 'batch_size': 16, 'num_train_epochs': 10, 'weight_decay': 0.05405722532491863}. Best is trial 7 with value: 0.788119602925087.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: T

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.291412,0.932418,0.810410,0.771044,0.783649
2,No log,0.300298,0.931868,0.803661,0.774411,0.783800
3,No log,0.301094,0.932967,0.817532,0.771044,0.785590
4,No log,0.301410,0.932418,0.811900,0.771044,0.784486


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


[I 2024-12-10 06:12:15,179] Trial 9 finished with value: 0.7844859495119011 and parameters: {'learning_rate': 3.5902662104696834e-06, 'batch_size': 8, 'num_train_epochs': 4, 'weight_decay': 0.00022329399786593472}. Best is trial 7 with value: 0.788119602925087.


Best trial:
FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.788119602925087], datetime_start=datetime.datetime(2024, 12, 10, 5, 52, 2, 109251), datetime_complete=datetime.datetime(2024, 12, 10, 6, 1, 28, 424710), params={'learning_rate': 4.805816125119564e-05, 'batch_size': 4, 'num_train_epochs': 9, 'weight_decay': 0.028066253106092263}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=7, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training time: {training_time:.2f} seconds")
print("<<==============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 9
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1170


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.329009,0.919780,0.775373,0.791246,0.774063
2,No log,0.352578,0.928571,0.791423,0.771044,0.773520
3,No log,0.345351,0.930220,0.805924,0.760943,0.769411
4,0.037000,0.454477,0.913736,0.796469,0.723906,0.734773
5,0.037000,0.359146,0.925824,0.803315,0.740741,0.763015
6,0.037000,0.346995,0.929670,0.791725,0.774411,0.776381
7,0.037000,0.364047,0.929670,0.803311,0.771044,0.774955
8,0.009500,0.335355,0.929121,0.790172,0.777778,0.779579
9,0.009500,0.328025,0.932967,0.800097,0.781145,0.787527


***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch

<<=================Training Time================>>
Training time: 503.24 seconds
<<==============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 260
  Batch size = 8


{'eval_loss': 0.32802528142929077, 'eval_accuracy': 0.932967032967033, 'eval_precision': 0.8000971113031433, 'eval_recall': 0.7811447811447811, 'eval_f1': 0.7875266369671817, 'eval_runtime': 3.654, 'eval_samples_per_second': 71.155, 'eval_steps_per_second': 9.031, 'epoch': 9.0}


In [14]:
!mkdir 'best_model_roberta-large_pharo'
!mkdir 'best_model_roberta-large_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_roberta-large_pharo')
tokenizer.save_pretrained('./best_model_roberta-large_pharo_tokenizer')

# Load model and tokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained('./best_model_roberta-large_pharo')
tokenizer = RobertaTokenizer.from_pretrained('./best_model_roberta-large_pharo_tokenizer')

Saving model checkpoint to ./best_model_roberta-large_pharo
Configuration saved in ./best_model_roberta-large_pharo/config.json
Model weights saved in ./best_model_roberta-large_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_roberta-large_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_roberta-large_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_roberta-large_pharo/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,


## Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

print("<<==============================================>>")
# Display metrics and inference time
print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 289
  Batch size = 8


<<==============================================>>
<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.9174493326742462, 'precision': 0.7321452049501617, 'recall': 0.7375415282392026, 'f1': 0.733647997728607}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 4.17 seconds
<<==============================================>>


In [18]:
!zip -r best_model_roberta-large_pharo.zip './best_model_roberta-large_pharo'
!zip -r best_model_roberta-large_pharo_tokenizer.zip './best_model_roberta-large_pharo_tokenizer'

  adding: best_model_roberta-large_pharo/ (stored 0%)
  adding: best_model_roberta-large_pharo/training_args.bin (deflated 51%)
  adding: best_model_roberta-large_pharo/pytorch_model.bin (deflated 14%)
  adding: best_model_roberta-large_pharo/config.json (deflated 55%)
  adding: best_model_roberta-large_pharo_tokenizer/ (stored 0%)
  adding: best_model_roberta-large_pharo_tokenizer/tokenizer_config.json (deflated 80%)
  adding: best_model_roberta-large_pharo_tokenizer/vocab.json (deflated 63%)
  adding: best_model_roberta-large_pharo_tokenizer/special_tokens_map.json (deflated 84%)
  adding: best_model_roberta-large_pharo_tokenizer/merges.txt (deflated 53%)


In [20]:
# Transfer the model to Google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_roberta-large_pharo.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo/"
!cp best_model_roberta-large_pharo_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo/best_model_roberta-large_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo_Tokenizer/best_model_roberta-large_pharo_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo/best_model_roberta-large_pharo.zip
replace ./best_model_roberta-large_pharo/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_pharo/training_args.bin  
  inflating: ./best_model_roberta-large_pharo/pytorch_model.bin  
  inflating: ./best_model_roberta-large_pharo/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_pharo_Tokenizer/best_model_roberta-large_pharo_tokenizer.zip
replace ./best_model_roberta-large_pharo_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_pharo_tokenizer/tokenizer_config.json  
  inflating: ./best_model_roberta-large_pharo_tokenizer/vocab.json  
  inflating: ./best_model_roberta-large_pharo_tokenizer/special_tokens_map.json  
  inflating: ./best_model_roberta-large_pharo_tokenizer/merges.txt  


In [23]:
best_model_roberta_large_pharo = 'best_model_roberta-large_pharo'
best_model_roberta_large_pharo_tokenizer = 'best_model_roberta-large_pharo_tokenizer'

In [24]:
# Load the model and tokenizer from the checkpoint
model = RobertaForSequenceClassification.from_pretrained(best_model_roberta_large_pharo)
tokenizer = RobertaTokenizer.from_pretrained(best_model_roberta_large_pharo_tokenizer)

text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")
output

loading configuration file best_model_roberta-large_pharo/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_

<<=================Inference Time================>>
Inference Time: 0.31 seconds
<<==============================================>>


SequenceClassifierOutput(loss=None, logits=tensor([[-7.8574, -5.8814, -5.3322, -7.0805, -7.1509,  6.2668, -8.0007]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 6
